<a href="https://colab.research.google.com/github/obraia/xgboost-tdc/blob/main/XGBoost_TDC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<header>
  <img src="https://upload.wikimedia.org/wikipedia/commons/c/c3/Python-logo-notext.svg" width="100" align="left" hspace="10px" vspace="0px"/>
  
  <h1><b>XGBoost</b> - TDC</h1>
  <h4>Bryan Diniz, Débora Oliveira, Isabela Fonseca e Thais Lorentz</h4>
</header>

<p>e<b>X</b>treme <b>G</b>radient <b>Boost</b>ing</p>

<h3><b>1 - Importação das bibliotecas necessárias</b></h3>

<p></p>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from xgboost import XGBClassifier
from xgboost import plot_tree
from google.colab import files
from io import BytesIO

print('Bibliotecas importadas!')

Bibliotecas importadas!


<h3><b>2 - Importação do dataset</b></h3>

<p>É possível carregar o arquivo .csv localmente.</p>

In [ ]:
arquivos = files.upload()
nomeArquivo = list(arquivos.keys())[0]

if not nomeArquivo.endswith('.csv'):
  raise Exception('Tipo de arquivo inválido!')

arquivo = arquivos[nomeArquivo]

df = pd.read_csv(BytesIO(arquivo))

print('Dataset "{nome}" importado!'.format(nome=nomeArquivo))

<h3><b>3 - Exibição das 5 primeiras instâncias do dataset</b></h3>

<p></p>

In [ ]:
df.head()

<h3><b>4 - Exibição de informações dos atributos do dataset</b></h3>

<p></p>

In [ ]:
df.info()

<h3><b>5 - Pré-processamento dos dados</b></h3>

<p>Os dados do dataset serão processado de forma que fique compatível com o algorítmo XGBoost, e buscando os melhores resultados</p>

<h4><b>5.1 - Tratamento de valores ausentes</b></h4>

<p></p>

In [ ]:
def print_valores_ausentes():
  print(df.isnull().sum())

print_valores_ausentes()

<h5><b>5.1.2 - Exclusão de instâncias com valores ausentes</b></h5>

<p></p>

In [ ]:
df.dropna(inplace=True)

print_valores_ausentes()

<h5><b>5.1.3 - Preenchimento de valores ausentes com a média</b></h5>

<p></p>

In [ ]:
media_temperature = round(df.temperature.mean())

df.temperature.fillna(media_temperature, inplace=True)

<h5><b>5.1.4 - Preenchimento de valores ausentes com a mediana</b></h5>

<p></p>

In [ ]:
mediana_temperature = df.temperature.median()

df.temperature.fillna(mediana_temperature, inplace=True)

<h5><b>5.1.5 - Preenchimento de valores ausentes com a moda (Valores que mais se repetem)</b></h5>

<p></p>

In [ ]:
moda_star_color = df.star_color.mode()[0]

df.star_color.fillna(moda_star_color, inplace=True)

<h4><b>5.2 - Converter variáveis categóricas para numéricas</b></h4>

<p>Como o algoritmo XGBoost só trabalha com valores númericos, os próximos passos serão de transformações dos atributos</p>

<h5><b>5.2.1 - Conversão de atributos categóricos para atributos simbólicos</b></h5>

<p>Nessa etapa iremos transformar os atributos <b>star_color</b> e <b>spectral_class</b> usando o conceito de <b>conversão simbólico-numérico</b></p>

In [ ]:
df_2 = pd.get_dummies(df, columns=['star_color'], drop_first=True)
df_2 = pd.get_dummies(df_2, columns=['spectral_class'], drop_first=True)

df_2.head()

<h5><b>5.2.1 - Conversão da classe categórica para numérica</b></h5>

<p>Diferente da etapa anterior, nossa classe pode manter uma relação númerica, isso não trará problemas na classificação, logo vamos usar uma conversão simples para atender ao algoritmo</p>

In [ ]:
df_3 = df_2.copy()

# transformação da nossa classe para uma variável category do pandas e conversão para inteiro
df_3['star_type'] = df_3['star_type'].astype('category').cat.codes


df_3.head()

<h5><b>5.2.2 - Visualização de como ficou o dataset após os tratamentos</b></h5>

<p></p>

In [ ]:
df_3.info()

<h5><b>5.2.3 Divisão do dataset</b></h5>

<p>Etapa onde os dados serão separados em conjuntos de treino, testes e a classe</p>

In [ ]:
y = df_3['star_type']
x = df_3.drop(['star_type'], axis=1)

# dividir entre conjuntos de treino e teste
train_x, test_x, train_y, test_y = train_test_split(x.values, y.values, test_size=0.33, random_state=7)

<h3><b>6 - Implementação do XGBoost</b></h3>

<p></p>

<h4><b>6.1 Inicio do treino e predição do modelo</b></h4>

<p>Ao aumentar o número de árvores <b>(n_estimators)</b> no modelo, é recomendado que diminua a taxa de aprendizado <b>(learning_rate)</b>, para que cada árvore não tenha uma grande influência e assim causando overfitting</p>

In [ ]:
model = XGBClassifier(learning_rate=0.01, n_estimators=700, max_depth=5, colsample_bynode=1, random_state=0)
 
model.fit(train_x, train_y)
 
y_pred = model.predict(test_x)
predictions = [round(value) for value in y_pred]

absolute_error = mean_absolute_error(test_y, predictions)
squared_error = mean_squared_error(test_y, predictions)
accuracy = accuracy_score(test_y, predictions)

print(np.sqrt(squared_error))
print("Erro médio absoluto: {:.4f}".format(absolute_error))
print("Erro quadrático médio: {:.4f}".format(squared_error))
print("Accuracy: {:.2f}%".format(accuracy * 100.0))

<h4><b>6.2 Avalidando o modelo</b></h4>

<p></p>

<h5><b>6.2.1 Matriz de confusão</b></h5>

<p></p>

In [ ]:
tipos_estrelas = df.star_type.unique()

fig, ax = plt.subplots(figsize=(10, 10))
plot_confusion_matrix(model, test_x, test_y, display_labels=tipos_estrelas, cmap=plt.cm.YlGn, ax=ax)  

<h5><b>6.2.2 Matriz de correlação</b></h5>

<p></p>

In [ ]:
df_4 = df.copy()

df_4['star_type'] = df_4['star_type'].astype('category').cat.codes
df_4['star_color'] = df_4['star_color'].astype('category').cat.codes
df_4['spectral_class'] = df_4['spectral_class'].astype('category').cat.codes

correlacao = df_4.corr()

fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(correlacao, annot=True, cmap=plt.cm.YlGn, square=True, ax=ax)
plt.show()

<h5><b>6.2.3 Importância dos atributos</b></h5>

<p></p>

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

plot_tree(model, num_trees=1, rankdir='LR')